In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
from __future__ import print_function
from dataGen import *
import pandas as pd
import csv
from six.moves import cPickle as pickle
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# import job's word2vec.csv
import pickle
job_vec = pd.read_csv("/mnt/Wendy/job_vector.csv", sep=",")
job_vec.head()

In [ ]:
# rename column name
job_vec1 = job_vec.rename(columns= {'jobno':'jobNo'})

In [2]:
# import pre-processed datamart
ratio = pd.read_csv("pop_all.csv")
ratio.head()

,uid,jobNo,applyJob,saveJob,viewJob,viewJob_sum,saveJob_sum,r_view,r_save,invoice,...,s2,s3,addr_no,s9,need_emp,need_emp1,startby,exp_jobcat1,exp_jobcat2,exp_jobcat3
0,2,208337,NaN,1.0,2.0,76.0,4.0,0.026316,0.25,5637971,...,0,0,6001002016,1,0,0.0,3,0,0,0
1,2,896465,NaN,NaN,1.0,76.0,4.0,0.013158,NaN,43816068,...,0,15,6001005013,1,1,1.0,3,0,0,0
2,2,1463885,NaN,NaN,1.0,76.0,4.0,0.013158,NaN,20954200,...,0,0,6001002020,1,1,1.0,2,0,0,0
3,2,3553643,NaN,NaN,1.0,76.0,4.0,0.013158,NaN,20954200,...,0,15,6001002020,1,1,1.0,3,0,0,0
4,2,4026316,NaN,1.0,2.0,76.0,4.0,0.026316,0.25,89400818,...,0,0,6001016010,11,1,1.0,1,0,0,0


In [3]:
# choose only ratio columns
ratio1 = ratio[['uid','jobNo','applyJob','saveJob','viewJob','r_view','r_save']]
ratio1.head()

,uid,jobNo,applyJob,saveJob,viewJob,r_view,r_save
0,2,208337,NaN,1.0,2.0,0.026316,0.25
1,2,896465,NaN,NaN,1.0,0.013158,NaN
2,2,1463885,NaN,NaN,1.0,0.013158,NaN
3,2,3553643,NaN,NaN,1.0,0.013158,NaN
4,2,4026316,NaN,1.0,2.0,0.026316,0.25


In [5]:
# processing step: ratio
ratio1['applyJob'] = ratio1['applyJob']/ratio1['applyJob']

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# delete temporary data to release memory
ratio.drop(ratio.index,inplace=True)

In [ ]:
# merge 300 columns
join300 =  pd.merge(ratio1 , job_vec1 , how='left', on=['jobNo'],indicator=True)
join300.head()

In [13]:
# training data
train_new = ratio1.sample(frac = 0.7)

In [14]:
# test data
t1 = pd.merge(ratio , train_new , how='left', on=['jobNo','uid'],indicator=True)
less_data = t1[t1._merge!='both']
test_new = less_data[['uid','jobNo','applyJob_x','saveJob_x','viewJob_x','r_view_x','r_save_x']]

In [15]:
test_new = test_new.rename(columns = {'applyJob_x': 'applyJob', 'saveJob_x': 'saveJob', 'viewJob_x': 'viewJob', 'r_view_x': 'r_view', 'r_save_x': 'r_save'})

In [16]:
# sort_values
train_new = train_new.sort_values(['uid','jobNo'])
test_new = test_new.sort_values(['uid','jobNo'])

In [17]:
#train data 補值
X_train = train_new.fillna(0).drop(['applyJob','uid','jobNo'], axis=1)
Y_train = train_new.fillna(0)["applyJob"]

In [18]:
#檢查筆數與欄位數
X_test  = test_new.fillna(0).drop(['uid','jobNo','applyJob'], axis=1).copy()
Y_test = test_new.fillna(0)["applyJob"]
X_train.shape, Y_train.shape, X_test.shape, X_test.shape

((21943299, 4), (21943299,), (9404271, 4), (9404271, 4))

## Model buliding

In [19]:
# NN
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
nn.fit(X_train, Y_train)
Y_pred = nn.predict(X_test)
acc_nn = round(nn.score(X_train, Y_train) * 100, 2)
acc_nn

88.07

In [24]:
# save nn-model in pickle
import pickle
f_nn = open("nn.pkl", "wb")
pickle.dump(nn, f_nn)
# remember to close, or the model cann't run
f_nn.close()

In [ ]:
# Logistic Regression 
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

In [ ]:
# Logistic Regression_class weight=balanced
logreg = LogisticRegression(class_weight='balanced')
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

In [ ]:
# Decision Tree
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

In [ ]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

In [ ]:
#存成output
submission = pd.DataFrame({
        "uid": test_new["uid"],
        "jobNo": test_new["jobNo"],
        "applyJob": Y_pred
    })
submission.to_csv('submission2.csv', index=False)

# Scoring

In [20]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [21]:
def compute_measures(tp, fn, fp, tn):
     recall = tn / (tn + fp)
     precision = tn / (tn + fn)
     F1 = 2 * (precision * recall) / (precision + recall)
     accuracy = (tp + tn) / (tp + tn + fp + fn)
     return recall, precision, F1, accuracy

In [23]:
cm5 = np.zeros(2**2)
cm5 += confusion_matrix(Y_test, Y_pred).flatten()
recall, precision, f1score, accuracy = compute_measures(*cm5)
print(recall)
print(precision)
print(accuracy)
print(f1score)

0.230138165978
0.738176547961
0.88077119428
0.350883022783


In [ ]:
cm = np.zeros(2**2)
cm += confusion_matrix(Y_test, Y_pred).flatten()
recall, precision, f1score, accuracy = compute_measures(*cm)
print(recall)
print(precision)
print(accuracy)
print(f1score)

In [ ]:
#Logistic Regression_class weight=balanced
cm2 = np.zeros(2**2)
cm2 += confusion_matrix(Y_test, Y_pred).flatten()
recall, precision, f1score, accuracy = compute_measures(*cm2)
print(recall)
print(precision)
print(accuracy)
print(f1score)

In [ ]:
#Decision Tree
cm3 = np.zeros(2**2)
cm3 += confusion_matrix(Y_test, Y_pred).flatten()
recall, precision, f1score, accuracy = compute_measures(*cm3)
print(recall)
print(precision)
print(accuracy)
print(f1score)

In [ ]:
# Random Forest
cm4 = np.zeros(2**2)
cm4 += confusion_matrix(Y_test, Y_pred).flatten()
recall, precision, f1score, accuracy = compute_measures(*cm3)
print(recall)
print(precision)
print(accuracy)
print(f1score)

# Save model

In [ ]:
# save logistic regression model
import pickle
f_model = open("log1_model.pkl", 'wb')
pickle.dump(logreg, f_model)
f_model.close